# Visualiseren output parameters
In het vorige script hebben een functie gemaakt die elke parameter combinatie van de DBScan een u-score toekent. Dit hebben we gedaan om zicht te krijgen welke parameters we moeten selecteren om stoplocaties te clusteren met behulp van de DBScan. De output uit deze functie gaan we nu visualiseren, zodat we de beste combinatie van parameters kunnen selecteren en deze kunnen invoeren in de DBScan. 

In [ ]:
# Laden packages
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing
import seaborn as sns
# from sklearn.metrics import average_precision_score 
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import sklearn
import multiprocessing as mp
from scipy.interpolate import griddata
from matplotlib.cm import coolwarm

In [ ]:
# Laden output vorige script
output_df = pd.read_csv("sample_output.csv")

##### Voeg filters toe
Hier verwijderen we de rijen waarbij de output 1 ruis of 1 stop cluster was. 

In [ ]:
# filter om rij met 1 stop of 1 ruis cluster te verwijderen
output_df = output_df[output_df.count_cluster != 0]
output_df = output_df[output_df.count_ruis != 0]

## Filter om punten te verwijderen met een u-score boven 20.000
# output_df["u"] = np.minimum(output_df["u"], 20000)

## Visualiseer alleen punten tussen de waarde.. en ...
#output_df = output_df[(output_df["count_cluster"] > 400) & (output_df["count_cluster"] < 600)]

##### Visualiseer u-score: epsilon vs. minimal samples

In [ ]:
# Creëer heatmap van u-scores om zo de juiste parameters te kunnen selecteren. Waarbij epsilon op de x-as staat en minimal samples op y-as.
# creëer array's
x = np.array(output_df["eps"]).round(3)  # rond epsilon af om leesbaarheid x-as te verbeteren
y = np.array(output_df["min_samples"])
z = np.array(output_df["u"])

# creëer pivot tabel
results = pd.DataFrame.from_dict(np.array([x,y,z]).T)
results.columns = ['x','y','z']
pivotted = results.pivot('y','x','z')

# Visualiseer pivot tabel met seaborn heatmap
sns.set()
ax=sns.heatmap(pivotted, cmap="Spectral")
ax.invert_yaxis() # draai heatmap

In [ ]:
# Scatterplot count clusters en ruis
# u_plots = output_df.copy()
u_plot = pd.melt(output_df, id_vars=["u"], value_vars=["count_cluster", "count_ruis"])
sns.scatterplot(x="u", y="value", hue = "variable", data=u_plot)

In [ ]:
## Creëer contourplot u-scores: epsilon vs. minimal samples
%matplotlib inline

# functie die grid maakt
def plot_contour(x,y,z,resolution = 1000,contour_method='linear'):
    resolution = str(resolution)+'j'
    X,Y = np.mgrid[min(x):max(x):complex(resolution),   min(y):max(y):complex(resolution)]
    points = [[a,b] for a,b in zip(x,y)]
    Z = griddata(points, z, (X, Y), method=contour_method)
    return X,Y,Z

X,Y,Z = plot_contour(x,y,z,resolution = 1000,contour_method='linear')

# contourplot aanmaken
with plt.style.context("seaborn-pastel"):
    fig, ax = plt.subplots(figsize=(13,8))
    ax.scatter(x,y, color="black", linewidth=1, edgecolor="ivory", s=50)
    img = ax.contourf(X,Y,Z,100)
    fig.colorbar(img)